In [1]:
import os
import sys

def is_colab():
    return 'google.colab' in sys.modules

if is_colab():
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_PATH = '/content/drive/MyDrive/NBS_Project'
else:
    # 自动根据当前脚本定位项目根路径
    PROJECT_PATH = os.path.abspath(os.path.join(os.path.dirname(__file__), '..')) \
        if '__file__' in globals() else os.getcwd()

if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)

print(f"项目路径设置为: {PROJECT_PATH}")

项目路径设置为: c:\Users\Barytes\Desktop\NBS_project


In [2]:
from src.config import Config
from src.main import create_MDs,create_ESP
import src.baseline as baseline
import src.NBS as NBS
import src.model as model
import numpy as np

CONFIG_PATH = os.path.join(PROJECT_PATH, "configs", "test1.json")
config = Config(CONFIG_PATH)

# Create MDs with a specific seed for reproducibility
seed = 41
MDs = create_MDs(config, seed=seed)
# np.random.shuffle(MDs)  # Shuffle the MDs

for md in MDs:
    print(md.param)

esp = create_ESP(config, seed=seed)
print(esp.param)

{'s': 10, 'l': 400, 'cn': 2.0036944949797606e-12, 'Fn': 11184.383282690447, 'kn': 0.6030448723331675, 'omega_n': 1, 'Rn': 4.13472934445215}
{'s': 10, 'l': 400, 'cn': 3.4154627557952525e-12, 'Fn': 11763.722651118956, 'kn': 0.6005547151781037, 'omega_n': 1, 'Rn': 10.248845012378245}
{'s': 10, 'l': 400, 'cn': 2.329039400190971e-12, 'Fn': 12132.134540686906, 'kn': 0.4558846802194673, 'omega_n': 1, 'Rn': 6.716553285779903}
{'s': 10, 'l': 400, 'cn': 1.278081871246422e-12, 'Fn': 13819.930286313422, 'kn': 0.49440308019386203, 'omega_n': 1, 'Rn': 9.231363952354403}
{'s': 10, 'l': 400, 'cn': 3.432905838092195e-12, 'Fn': 13913.825949541608, 'kn': 0.5265274112699365, 'omega_n': 1, 'Rn': 7.948101841125762}
{'lambda0': 20, 'D0': 10, 'theta': 10, 'o': 10, 'omega_0': 2}


In [3]:
lam_swm, p_swm, Dmax_swm = baseline.social_welfare_maximization(esp, MDs)
print(f"SWM λ: {lam_swm}")
print(f"SWM p: {p_swm}")  
print(f"SWM Dmax: {Dmax_swm}")
r_swm = baseline.solve_r_NBP(esp, MDs, Dmax_swm, lam_swm, p_swm)
print(f"SWM r: {r_swm}")

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.04017705789245194
            Iterations: 97
            Function evaluations: 1458
            Gradient evaluations: 93
SWM λ: [1.87418807 1.9274377  4.73977224 4.94837847 4.58241755]
SWM p: [1.00000000e-06 1.00000008e-06 1.12495805e+04 1.20828069e+04
 1.06223758e+04]
SWM Dmax: 1.8849736036717633
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.004944714982477892
            Iterations: 37
            Function evaluations: 259
            Gradient evaluations: 37
SWM r: [4953644.80647307 4953644.806575   4953695.52693493 4953716.28199323
 4953725.61350551]


In [4]:
lam_opt, p_opt, r_opt, Dmax_opt = baseline.optimal_NBP(esp, MDs)
print(f"Optimal λ: {lam_opt}")
print(f"Optimal p: {p_opt}")  
print(f"Optimal r: {r_opt}")
print(f"Optimal Dmax: {Dmax_opt}")

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.014353182246047152
            Iterations: 5
            Function evaluations: 18
            Gradient evaluations: 1
Optimal λ: [20.  0. 20.  0.  0.]
Optimal p: [1.11843833e+04 1.00000000e-06 1.00000000e-06 1.00000000e-06
 1.39138259e+04]
Optimal r: [7.14496116e+13 3.49341043e+14 2.76492679e+15 1.82825513e+15
 1.71438576e+15]
Optimal Dmax: 1e-06


In [5]:
# 核对 Proposition 1
assert np.allclose(lam_swm, lam_opt, atol=1e-4)
assert np.allclose(p_swm,   p_opt,   atol=1e-4)
assert abs(Dmax_swm - Dmax_opt) < 1e-4
print("OK – SWM and NBP coincide; difference only in r.")

AssertionError: 

In [8]:
lamb_admm, p_admm, Dmax_admm = NBS.ADMM(esp, MDs)
print(f"ADMM λ: {lamb_admm}")
print(f"ADMM p: {p_admm}")  
print(f"ADMM Dmax: {Dmax_admm}")
r_admm = NBS.negotiation(esp, MDs,lamb_admm, p_admm, Dmax_admm)
print(f"ADMM r: {r_admm}")

SLSQP failed: Positive directional derivative for linesearch
SLSQP failed: Positive directional derivative for linesearch
SLSQP failed: Positive directional derivative for linesearch
SLSQP failed: Positive directional derivative for linesearch
SLSQP failed: Positive directional derivative for linesearch
SLSQP failed: Positive directional derivative for linesearch
SLSQP failed: Positive directional derivative for linesearch
SLSQP failed: Positive directional derivative for linesearch
SLSQP failed: Positive directional derivative for linesearch
SLSQP failed: Positive directional derivative for linesearch
SLSQP failed: Positive directional derivative for linesearch
SLSQP failed: Positive directional derivative for linesearch
SLSQP failed: Positive directional derivative for linesearch
SLSQP failed: Positive directional derivative for linesearch
SLSQP failed: Positive directional derivative for linesearch
SLSQP failed: Positive directional derivative for linesearch
SLSQP failed: Positive d

TypeError: unsupported operand type(s) for -: 'NoneType' and 'float'

In [9]:
print(model.nash_product_log(esp, MDs, lam_opt, p_opt, r_opt))
print(model.nash_product_log(esp, MDs, lam_swm, p_swm, r_swm))
print(model.nash_product_log(esp, MDs, lamb_admm, p_admm, r_admm))

TypeError: unsupported operand type(s) for -: 'NoneType' and 'float'